<p style="text-align: center;font-size: 40pt">Testing ICP</p>
<p style="text-align: center;font-size: 20pt">Annexe to the assigment</p>

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from scipy import spatial
from IPython.display import HTML

%run ./scripts/helper_func.py
path = "{0}/lessons/transformations_2d/scripts/helper_func.py".format(get_root_path())
%run $path
path = "{0}/common/scripts/style.py".format(get_root_path())
%run $path

In [ ]:
# Generate two point clouds
nb_pts = 400
P = build_room([1.2, 2.], [2.2, 1.5], angle=-0, nb_pts=nb_pts)
Q = build_room([1.8, 2.], [2.8, 2.2], angle=-0.3, nb_pts=nb_pts)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,8))
draw_point_clouds(ax, P=P, Q=Q)

In [ ]:
# keep 75 % of the points
prob = np.random.uniform(size=P.shape[1])
mask = (prob < 0.75)
P = P[:, mask]

prob = np.random.uniform(size=Q.shape[1])
mask = (prob < 0.75)
Q = Q[:, mask]

normals_Q = np.zeros([2, Q.shape[1]])
k_nn = 20

# build kdtree
tree = spatial.KDTree(Q.T)
dist, indices = tree.query(Q.T, k=k_nn)

for i, nn_i in enumerate(indices):
    neighbors = Q[0:2, nn_i]
    mu = np.mean(neighbors, axis=1)
    errors = (neighbors.T - mu).T
    cov = 1/k_nn * (errors @ errors.T)
    eigenValues, eigenVectors = sorted_eig(cov)
    normals_Q[:,i] = eigenVectors[:,0] # smallest eigen vector

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,8))
draw_point_clouds(ax, Q=Q, normals_Q=normals_Q)

In [ ]:
nb_pts = P.shape[1]

tree = spatial.KDTree(Q.T)
dist, indices = tree.query(P.T, k=1)

errors = Q[:2, indices] - P[:2]
h = np.empty(nb_pts)
G = np.empty([3, nb_pts])

for i in range(nb_pts):
    q_id = indices[i]
    n = normals_Q[:,q_id]
    p = P[:,i]
    e = errors[:,i]
    h[i] = np.dot(e, n)
    cross = p[0] * n[1] - p[1] * n[0] # pseudo-cross product in 2D
    G[0:2,i] = n
    G[2,i] = cross


fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
ax=axs[0]
draw_point_clouds(ax, P=P, Q=Q, errors=errors)
ax.set_title("Point clouds")

ax=axs[1]
ax.quiver(np.zeros(nb_pts), np.zeros(nb_pts), errors[0], errors[1], 
          color="tab:red", alpha=0.2,
          angles='xy', scale_units='xy', scale=1.)
ax.set_aspect('equal', adjustable='box')
ax.set_xlabel(r"Error on $\vec{\mathscr{x}}$")
ax.set_ylabel(r"Error on $\vec{\mathscr{y}}$")
ax.set_title("Residual error")
lim = [-0.25, 0.25]
ax.set_xlim(lim)
ax.set_ylim(lim)
fig.tight_layout()

In [ ]:
x = np.linalg.solve(G @ G.T, G @ h) # this gives: [x, y, theta]

T = rigid_tranformation(x)
P_prime = T @ P

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,6))
draw_point_clouds(ax, P=P_prime, Q=Q, T=T)